**Reference: https://python.langchain.com/v0.1/docs/use_cases/sql/large_db/**

What happens in this notebook:

### **Table Model Definition**
   - **`Table` Class**: This is a simple Pydantic model representing a SQL table. It has one attribute, `name`, which is a string and is described as "Name of table in SQL database."
     - This model is used in the extraction process to match relevant SQL tables based on the user's query.

### **Helper Function - `get_tables`**
   - **`get_tables`**: This function takes a list of `Table` objects (i.e., categories such as "Music" or "Business") and returns a list of corresponding SQL table names based on the category.
     - For example, if the category is `"Music"`, the tables `"Album"`, `"Artist"`, `"Genre"`, etc., are added to the result.
     - Similarly, for `"Business"`, the corresponding tables like `"Customer"`, `"Employee"`, etc., are included.

### **Designing the agent for the large DB**

- **Step 1: Initialize LLM (`sql_agent_llm`)**: The LLM is instantiated with a given model (e.g., `"gpt-3.5-turbo"`) and temperature. The temperature controls how creative/random the model's responses are.
- **Step 2: Connect to the SQL Database (`db`)**: The connection to the Chinook SQLite database is established. The database URI is constructed using the `sqldb_directory` provided.
- **Step 3: Define Category Chain (`category_chain`)**: The `category_chain_system` is defined, which is a string explaining the categories available (like "Music" and "Business"). This chain determines which SQL tables are relevant to the user query based on the category.
- **Step 4: Chain Creation**:
- **`category_chain`**: This uses the `create_extraction_chain_pydantic` function, which creates an extraction chain that identifies relevant SQL tables from the user's question using the `Table` Pydantic model and the LLM.
- **`table_chain`**: A chain is formed by combining the output from `category_chain` with the `get_tables` function, so it maps categories to the actual SQL tables.
- **Step 5: Query Chain (`query_chain`)**: This creates a SQL query chain using the LLM and the database (`self.db`). It takes the SQL tables and constructs a query.
- **Step 6: Table Chain Input Handling**: The `"question"` key from the user input is mapped to the `"input"` key expected by the `table_chain`. This enables the chain to process user queries correctly.
- **Step 7: Full Chain Construction**: Finally, the full chain (`full_chain`) is created by combining:
1. **`RunnablePassthrough.assign`**: This sets up a step that assigns the `table_names_to_use` using the result of the `table_chain`.
2. **`query_chain`**: Executes the SQL query once the relevant tables are identified.

In [1]:
import os
from dotenv import load_dotenv
from pyprojroot import here
from typing import List
from langchain_community.utilities import SQLDatabase
from langchain_openai import ChatOpenAI
from pprint import pprint

load_dotenv()

True

**Set the environment variables and load the LLM**

In [2]:
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

sql_agent_llm = ChatOpenAI(model="gpt-4", temperature=0)
table_extractor_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# llm = ChatOpenAI(model="gpt-4o")

In [3]:
sqldb_directory = here("data/ICDD-energy.db")
db = SQLDatabase.from_uri(f"sqlite:///{sqldb_directory}")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Comments LIMIT 10;")

sqlite
['Applications', 'Comments', 'Compositions', 'Melting Points']


"[(353, 10359, None, None, 'Decomposes at 1750 C'), (494, 10503, None, None, None), (569, 10579, None, None, None), (618, 10628, None, None, 'Transition to calcite at 520. Antacid'), (636, 10646, None, 'A new method of X-Ray crystal analysis', None), (667, 10677, None, None, None), (696, 10706, None, None, None), (738, 10750, None, None, None), (780, 10792, None, None, None), (784, 10796, None, None, None)]"

**Prepare the `Table` class**

In [4]:
#from langchain_core.pydantic_v1 import BaseModel, Field
from pydantic import BaseModel, Field

class Table(BaseModel):
    """
    Represents a table in the SQL database.

    Attributes:
        name (str): The name of the table in the SQL database.
    """
    name: str = Field(description="Name of table in SQL database.")

### **Strategy A:**

In [5]:
table_names = "\n".join(db.get_usable_table_names())
print(table_names)

Applications
Comments
Compositions
Melting Points


In [13]:
from langchain.chains.openai_tools import create_extraction_chain_pydantic

system = f"""Return the names of ALL the SQL tables that MIGHT be relevant to the user question. \
The tables are:

{table_names}

Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""
table_chain = create_extraction_chain_pydantic(pydantic_schemas=Table, llm=table_extractor_llm, system_message=system)
table_chain.invoke({"input": "how many compositions are there in the database?"})

[Table(name='Compositions')]

### **Strategy B:**

Music:

- "Album"
- "Artist"
- "Genre"
- "MediaType"
- "Playlist"
- "PlaylistTrack"
- "Track"

Business:

- "Customer"
- "Employee"
- "Invoice"
- "InvoiceLine"

In [22]:
from langchain.chains.openai_tools import create_extraction_chain_pydantic

system = f"""You will recieve a question.

If the question is about **Compositions**, return **ALL** these tables:
  - "Compositions"

If the question is about **Applications**, return **ALL** these tables:
  - "Applications"

If the question is about **Melting Points**, return **ALL** these tables:
  - "Melting Points"

If the question is about anything other than **Compositions**, **Applications**, and **Melting Points**, return **ALL** these tables:
  - "Comments"
if you don't know, return all tables.
The tables are:
 -"Compositions"
-"Applications"
-"Melting Points"
-"Comments"
Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed.
."""
table_chain = create_extraction_chain_pydantic(pydantic_schemas=Table, llm=table_extractor_llm, system_message=system)
table_chain.invoke({"input": "where most articles have been sited in the database?"})

[Table(name='Comments'),
 Table(name='Compositions'),
 Table(name='Applications'),
 Table(name='Melting Points')]

### **Strategy C:**

- **Step 1: Define the category**

In [28]:
from langchain.chains.openai_tools import create_extraction_chain_pydantic

system = """Return the names of the SQL tables that are relevant to the user question. \
The tables are:

Compositions
Applications
Melting Points
Comments
Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""
category_chain = create_extraction_chain_pydantic(pydantic_schemas=Table, llm=table_extractor_llm, system_message=system)

In [29]:
category_chain.invoke({"input": "What is the most frequently cited composition in the database?"})

[Table(name='Compositions'),
 Table(name='Applications'),
 Table(name='Melting Points'),
 Table(name='Comments')]

- **Step 2: Execute the python function**

In [33]:
def get_tables(categories: List[Table]) -> List[str]:
    """Maps category names to corresponding SQL table names.

    Args:
        categories (List[Table]): A list of `Table` objects representing different categories.

    Returns:
        List[str]: A list of SQL table names corresponding to the provided categories.
    """
    tables = []
    for category in categories:
        if category.name == "Compositions":
            tables.extend(
                ["Compositions"]
            )
        elif category.name == "Melting Points":
            tables.extend(["Melting Points"])
        elif category.name == "Applications":
            tables.extend(["Applications"])
        else:
            tables.extend(["Comments"])
    return tables


table_chain = category_chain | get_tables 
table_chain.invoke({"input": "What is the most frequently cited composition in the database?"})

['Compositions', 'Applications', 'Melting Points', 'Comments']

### **Final step:**

**Attach the desired strategy to your SQL agent**

In [34]:
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import create_sql_query_chain
from operator import itemgetter

query_chain = create_sql_query_chain(sql_agent_llm, db)
# Convert "question" key to the "input" key expected by current table_chain.
table_chain = {"input": itemgetter("question")} | table_chain
# Set table_names_to_use using table_chain.
full_chain = RunnablePassthrough.assign(table_names_to_use=table_chain) | query_chain

**Test the agent**

In [38]:
query = full_chain.invoke(
    {"question": "What are all unique application values in the database?"}
)
print(query)

SELECT DISTINCT "Description" FROM "Applications"


In [39]:
db.run(query)

"[('Battery Material',), ('Intercalate',), ('Ionic Conductor',), ('Superconducting Material',), ('Superconducting Material (Superconductor Reaction Product)',), ('Superconducting Material (Conventional Superconductor)',), ('Superconducting Material (Superconductor Related Material)',), ('Superconducting Material (Superconductor Reaction Product, Superconductor Related Material)',), ('Superconducting Material (High Tc Superconductor)',), ('Superconducting Material (Superconductor Related Material, High Tc Superconductor)',), ('Superconducting Material (Conventional Superconductor, High Tc Superconductor)',), ('Superconducting Material (Conventional Superconductor, Superconductor Related Material)',)]"

**Prepare the tool (Don't run the following cell)**

In [ ]:
# class ChinookSQLAgent:
#     """
#     A specialized SQL agent that interacts with the Chinook SQL database using an LLM (Large Language Model).

#     The agent handles SQL queries by mapping user questions to relevant SQL tables based on categories like "Music"
#     and "Business". It uses an extraction chain to determine relevant tables based on the question and then
#     executes queries against the database using the appropriate tables.

#     Attributes:
#         sql_agent_llm (ChatOpenAI): The language model used for interpreting and interacting with the database.
#         db (SQLDatabase): The SQL database object, representing the Chinook database.
#         full_chain (Runnable): A chain of operations that maps user questions to SQL tables and executes queries.

#     Methods:
#         __init__: Initializes the agent by setting up the LLM, connecting to the SQL database, and creating query chains.

#     Args:
#         sqldb_directory (str): The directory where the Chinook SQLite database file is located.
#         llm (str): The name of the LLM model to use (e.g., "gpt-3.5-turbo").
#         llm_temperature (float): The temperature setting for the LLM, controlling the randomness of responses.
#     """

#     def __init__(self, sqldb_directory: str, llm: str, llm_temerature: float) -> None:
#         """Initializes the ChinookSQLAgent with the LLM and database connection.

#         Args:
#             sqldb_directory (str): The directory path to the SQLite database file.
#             llm (str): The LLM model identifier (e.g., "gpt-3.5-turbo").
#             llm_temerature (float): The temperature value for the LLM, determining the randomness of the model's output.
#         """
#         self.sql_agent_llm = ChatOpenAI(
#             model=llm, temperature=llm_temerature)

#         self.db = SQLDatabase.from_uri(f"sqlite:///{sqldb_directory}")
#         print(self.db.get_usable_table_names())
#         category_chain_system = """Return the names of the SQL tables that are relevant to the user question. \
#         The tables are:

#         Music
#         Business"""
#         category_chain = create_extraction_chain_pydantic(
#             Table, self.sql_agent_llm, system_message=category_chain_system)
#         table_chain = category_chain | get_tables  # noqa
#         query_chain = create_sql_query_chain(self.sql_agent_llm, self.db)
#         # Convert "question" key to the "input" key expected by current table_chain.
#         table_chain = {"input": itemgetter("question")} | table_chain
#         # Set table_names_to_use using table_chain.
#         self.full_chain = RunnablePassthrough.assign(
#             table_names_to_use=table_chain) | query_chain


# @tool
# def query_chinook_sqldb(query: str) -> str:
#     """Query the Chinook SQL Database. Input should be a search query."""
#     # Create an instance of ChinookSQLAgent
#     agent = ChinookSQLAgent(
#         sqldb_directory=TOOLS_CFG.chinook_sqldb_directory,
#         llm=TOOLS_CFG.chinook_sqlagent_llm,
#         llm_temerature=TOOLS_CFG.chinook_sqlagent_llm_temperature
#     )

#     query = agent.full_chain.invoke({"question": query})

#     return agent.db.run(query)